# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
print(y)

0       0
1       0
2       0
3       1
4       0
       ..
1594    0
1595    1
1596    1
1597    0
1598    1
Name: credit_ranking, Length: 1599, dtype: int64


In [6]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns="credit_ranking")


# Review the features DataFrame
X

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance
X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [9]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
input_nodes

11

In [10]:
# Define the number of hidden nodes for the first hidden layer
first_layer_neurons = 6

# Define the number of hidden nodes for the second hidden layer
second_layer_neurons = 3

# Define the number of neurons in the output layer
output_neurons = 1

In [11]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(units=first_layer_neurons, activation="relu", input_dim= input_nodes))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=second_layer_neurons, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [12]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 3)                 21        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97
Trainable params: 97
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [13]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 [==============================] - 3s 5ms/step - loss: 0.6736 - accuracy: 0.5696
Epoch 2/50
38/38 [==============================] - 0s 5ms/step - loss: 0.6642 - accuracy: 0.6397
Epoch 3/50
38/38 [==============================] - 0s 6ms/step - loss: 0.6551 - accuracy: 0.6814
Epoch 4/50
38/38 [==============================] - 0s 6ms/step - loss: 0.6467 - accuracy: 0.6939
Epoch 5/50
38/38 [==============================] - 0s 5ms/step - loss: 0.6390 - accuracy: 0.6989
Epoch 6/50
38/38 [==============================] - 0s 4ms/step - loss: 0.6319 - accuracy: 0.7098
Epoch 7/50
38/38 [==============================] - 0s 4ms/step - loss: 0.6255 - accuracy: 0.7173
Epoch 8/50
38/38 [==============================] - 0s 4ms/step - loss: 0.6193 - accuracy: 0.7164
Epoch 9/50
38/38 [==============================] - 0s 4ms/step - loss: 0.6139 - accuracy: 0.7231
Epoch 10/50
38/38 [==============================] - 0s 4ms/step - loss: 0.6084 - accuracy: 0.7281
Epoch 11/50
38/38 [

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [14]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
nn_model.loss, nn_model.accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {nn_model.loss}, Accuracy: {nn_model.accuracy}")

13/13 - 1s - loss: 0.7218 - accuracy: 0.4700 - 790ms/epoch - 61ms/step
Loss: 0.7217989563941956, Accuracy: 0.4699999988079071


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [15]:
# Set the model's file path
file_path = Path("models/student_loan_keras_model.keras")

# Export your model to a keras file
nn_model.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [16]:
# Set the model's file path
file_path = Path("models/student_loan_keras_model.keras")

# Load the model to a new object
loaded_model = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [17]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled, verbose=2)

# Display a sample of the predictions
predictions

13/13 - 0s - 193ms/epoch - 15ms/step


array([[0.35733804],
       [0.48947614],
       [0.3616058 ],
       [0.4118398 ],
       [0.3168504 ],
       [0.5       ],
       [0.45323673],
       [0.5       ],
       [0.33686838],
       [0.5       ],
       [0.2791174 ],
       [0.5       ],
       [0.4046789 ],
       [0.5       ],
       [0.48667896],
       [0.3868617 ],
       [0.34367925],
       [0.5       ],
       [0.37846652],
       [0.5       ],
       [0.5       ],
       [0.48185304],
       [0.5       ],
       [0.5       ],
       [0.4973921 ],
       [0.3175297 ],
       [0.46277252],
       [0.40158874],
       [0.46894675],
       [0.02779939],
       [0.49125594],
       [0.5       ],
       [0.4416559 ],
       [0.4567836 ],
       [0.32700405],
       [0.32484394],
       [0.5       ],
       [0.5       ],
       [0.5       ],
       [0.5       ],
       [0.4850595 ],
       [0.28964636],
       [0.44622222],
       [0.28881347],
       [0.28797114],
       [0.5       ],
       [0.5       ],
       [0.5  

In [24]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(columns = ['predictions'], data=predictions)
predictions_df['predictions'] = round(predictions_df['predictions'], 0)
predictions_df.head()

,predictions
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


### Step 4: Display a classification report with the y test data and predictions

In [25]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["predictions"].values))

              precision    recall  f1-score   support

           0       0.47      1.00      0.64       188
           1       0.00      0.00      0.00       212

    accuracy                           0.47       400
   macro avg       0.23      0.50      0.32       400
weighted avg       0.22      0.47      0.30       400



c:\Users\vuvee\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vuvee\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vuvee\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

